# load data

In [9]:
import json
import sqlite3
import pandas as pd
import re
from urllib.parse import urlparse
import requests

In [10]:
from datetime import date
from datetime import timedelta

## load tweets older then two weeks

In [11]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
    
from pipeline_functions import load_tweets


In [12]:
df_tweets = load_tweets("../home.db", days=0)
df_tweets = df_tweets[df_tweets.retweeted_status == "N/A"] # remove RT

# utils

In [13]:
df_tweets.tail()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
86014,1358203060363870209,2660596250,Joint Faculty Position at NYU (Center for Neur...,2021-02-06T23:56:58+00:00,en,N/A,N/A,0,N/A
86015,1358203122045263873,2324423269,Reviewer #3 is unimpressed https://twitter.com...,2021-02-06T23:57:13+00:00,en,N/A,N/A,0,N/A
86016,1358203205541376000,6035812,ELABORATION IS A FORM OF POLLUTION #BarbaraHol...,2021-02-06T23:57:33+00:00,en,N/A,N/A,0,N/A
86017,1358203534714413056,19734656,"@PGSchuermann “From standpoint of learning, k...",2021-02-06T23:58:51+00:00,en,N/A,N/A,0,1358202579008724992
86018,1358203606806241280,331639736,pet rat says hi https://twitter.com/rishmishra...,2021-02-06T23:59:08+00:00,en,N/A,N/A,0,N/A


In [14]:
from requests_html import HTMLSession
url = "https://lukecarneal.substack.com/p/why-are-some-organic-farmers-turning"
print(HTMLSession().get(url).html.find('title', first=True).text)

Why Are Some Organic Farmers Turning to Reactionary Politics? - The Farm Worker Bulletin


In [15]:
def grab_title(url):
    try:
        title = HTMLSession().get(url).html.find('title', first=True).text
        return title
    except Exception as ex:
        return 0

In [16]:
url = "https://buff.ly/2T4jips"
grab_title(url)

0

In [17]:
# może da się to użyć z pandas i będzie szybość?

In [18]:
def expand_link(url):
    if url in ["https://", "http://"]:
        return url
    else:
        try:
            session = requests.Session()
            resp = session.head(url, allow_redirects=True)
            long_url = resp.url
            return long_url
        except Exception as ex:
            return url

In [19]:
from pipeline_functions import remove_tw_urls, find_url, get_domain, remove_empty_str
def find_news(df, news_domains_list):
    df["clean_text"] = df["full_text"].apply(remove_tw_urls)
    df["urls"] = df["clean_text"].apply(find_url)
    df.drop(["clean_text"], axis=1, inplace=True)
#     df["urls"] = df.urls.apply(lambda x: [clean_links(d) for d in x])
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
#     df.drop(["urls"], axis=1, inplace=True)

    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())

    for col in new_columns_list:
        df[col] = df[col].isin(news_domains_list)

#     df.drop(["domains"], axis=1, inplace=True)

    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    df.drop(new_columns_list, axis=1, inplace=True)

    return df

In [20]:
def drop_contains(df, column_name, word_list):
    for string in word_list:
        df["lower"] = df["full_text"].str.lower()
        df = df[df["lower"].str.contains(string)]
        df.drop(["lower"], axis=1, inplace=True)
    return df

# searching for not-expanded news urls

In [21]:
dfz = df_tweets.copy()

In [22]:
with open("news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())

In [23]:
dfz = find_news(dfz, news_domains)

In [24]:
dfz["has_domain"] = dfz.domains.apply(lambda x: len(x))

In [25]:
dfz.head()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,urls,domains,contains_news,has_domain
0,5596520,713143,soo hungry need to find my wife and head to pf...,2007-02-20T02:43:19+00:00,en,N/A,N/A,0,N/A,[],[],0,0
1,154674522,7193842,Hofstadter’s Law: A task always takes longer t...,2007-07-17T20:45:46+00:00,en,N/A,N/A,0,N/A,[],[],0,0
2,412145582,4829901,Adobe Updater must update itself before instal...,2007-11-13T21:45:35+00:00,en,N/A,N/A,0,N/A,[],[],0,0
3,922321981,16298441,no,2008-09-15T17:25:20+00:00,und,N/A,N/A,0,N/A,[],[],0,0
4,2627602600,21454322,"Went on a USO trip to Guantanamo Bay, Cuba a f...",2009-07-14T05:15:27+00:00,en,N/A,N/A,0,N/A,[],[],0,0


In [26]:
# dfz["has_domain"] = dfz.domains.apply(lambda x: 1 if len(x) > 0 else 0)
dfz["has_domain"] = dfz.domains.apply(lambda x: len(x))

In [27]:
dfz = dfz[dfz.has_domain > 0]
dfz.shape

(18196, 13)

In [28]:
dfz = dfz[dfz.contains_news == 0]
dfz.shape

(17975, 13)

In [29]:
dfz.reset_index(drop=True, inplace=True)

In [30]:
dfz.head()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,urls,domains,contains_news,has_domain
0,177008089394970624,5110861,"In 1996 a man took a NZ radio station hostage,...",2012-03-06T12:29:51+00:00,en,N/A,N/A,0,N/A,[http://edition.cnn.com/WORLD/fringe/9603/03-2...,[edition.cnn.com],0,1
1,451159870306549761,159169312,Course slides from @Stanford and @stanfordsyms...,2014-04-02T00:51:09+00:00,en,N/A,N/A,0,N/A,[http://stanford.edu/~zdar/week1.pdf],[stanford.edu],0,1
2,574518676575162369,68132773,Robert Capa's Omaha Beach by Dominique Bertail...,2015-03-08T10:35:01+00:00,en,N/A,N/A,0,N/A,[http://bandedessinee.blog.lemonde.fr/2014/06/...,[bandedessinee.blog.lemonde.fr],0,1
3,627427921624481792,794010396,600cals per day for 7 days reverses T2 diabete...,2015-08-01T10:37:28+00:00,en,N/A,N/A,0,N/A,[http://link.springer.com/article/10.1007/s001...,[link.springer.com],0,1
4,681513454931345408,2895499182,Found out there is a Fake Kanji creation conte...,2015-12-28T16:34:04+00:00,en,N/A,N/A,0,N/A,[http://www.sankeisquare.com/event/kanjicontes...,[www.sankeisquare.com],0,1


In [31]:
new_columns_list = []
max_nr_dom = dfz.urls.str.len().max()
for i in range(max_nr_dom):
    new_columns_list.append(f"urls{i+1}")
dfz[new_columns_list] = pd.DataFrame(dfz.urls.tolist())
# for col in new_columns_list:
#     dfz[col] = dfz[col].isin(short_urls_list)
#     dfz["checked"] = dfz[new_columns_list].sum(axis=1)
#     dfz["checked"] = dfz.checked.apply(lambda x: x if x == 0 else 1)

In [32]:
# %%time
# dfz.urls1[:10].apply(grab_title)
# # Wall time: 8.21 s

In [33]:
# %%time
# dfz.urls1.apply(grab_title)

In [34]:

domain_list = dfz.domains.tolist()

In [35]:
# from requests_html import AsyncHTMLSession
# asession = AsyncHTMLSession()
# async def grab_title_async(url):
#     res = await asession.get(url)
#     title = res.html.find('title', first=True).text
#     print(title

In [36]:
dfz.shape

(17975, 19)

In [37]:
dfz["urls_exp"] = 0

In [38]:
# import json
# short_url =[]
# for c, i in enumerate(dfz.urls):
#     print(c)
#     if (c + 1) % 100 == 0:
#         print(f"{(c+1)} / {len(dfz.urls)}")
#     for d in i:
#         dd = get_domain(d)
#         e = expand_link(d)
#         de = get_domain(e)
#         if not (de == dd):
#             short_url.append({"short_url": dd, "long_url": de, "is_news": (de in news_domains)})
#     with open('short_url', 'w') as file:
#         file.write(json.dumps(short_url, indent=4))



In [39]:
with open('short_url') as json_file:
    short_url = json.load(json_file)

In [40]:
short_urls_list = pd.DataFrame(short_url).short_url.tolist()

In [41]:
short_urls_list[:10]

['bandedessinee.blog.lemonde.fr',
 'gifthefeeling.com',
 'kite.com',
 'ow.ly',
 'youtu.be',
 'differantly.com',
 'youtu.be',
 'buff.ly',
 'www2.philly.com',
 'bit.ly']

In [42]:
dfz[dfz.id == 574518676575162369]

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,urls,domains,contains_news,has_domain,urls1,urls2,urls3,urls4,urls5,urls6,urls_exp
2,574518676575162369,68132773,Robert Capa's Omaha Beach by Dominique Bertail...,2015-03-08T10:35:01+00:00,en,N/A,N/A,0,N/A,[http://bandedessinee.blog.lemonde.fr/2014/06/...,[bandedessinee.blog.lemonde.fr],0,1,http://bandedessinee.blog.lemonde.fr/2014/06/0...,None,None,None,None,None,0


In [43]:
new_columns_list = []
max_nr_dom = dfz.domains.str.len().max()
for i in range(max_nr_dom):
    new_columns_list.append(f"domain{i+1}")
dfz[new_columns_list] = pd.DataFrame(dfz.domains.tolist())
for col in new_columns_list:
    dfz[col] = dfz[col].isin(short_urls_list)
    dfz["checked"] = dfz[new_columns_list].sum(axis=1)
    dfz["checked"] = dfz.checked.apply(lambda x: x if x == 0 else 1)
dfz.drop(new_columns_list, axis=1, inplace=True)

In [44]:
new_columns_list

['domain1', 'domain2', 'domain3', 'domain4', 'domain5', 'domain6']

In [45]:
# dfz.drop(new_columns_list, axis=1, inplace=True)

In [46]:
dfz.head()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,urls,...,contains_news,has_domain,urls1,urls2,urls3,urls4,urls5,urls6,urls_exp,checked
0,177008089394970624,5110861,"In 1996 a man took a NZ radio station hostage,...",2012-03-06T12:29:51+00:00,en,N/A,N/A,0,N/A,[http://edition.cnn.com/WORLD/fringe/9603/03-2...,...,0,1,http://edition.cnn.com/WORLD/fringe/9603/03-27...,None,None,None,None,None,0,0
1,451159870306549761,159169312,Course slides from @Stanford and @stanfordsyms...,2014-04-02T00:51:09+00:00,en,N/A,N/A,0,N/A,[http://stanford.edu/~zdar/week1.pdf],...,0,1,http://stanford.edu/~zdar/week1.pdf,None,None,None,None,None,0,0
2,574518676575162369,68132773,Robert Capa's Omaha Beach by Dominique Bertail...,2015-03-08T10:35:01+00:00,en,N/A,N/A,0,N/A,[http://bandedessinee.blog.lemonde.fr/2014/06/...,...,0,1,http://bandedessinee.blog.lemonde.fr/2014/06/0...,None,None,None,None,None,0,1
3,627427921624481792,794010396,600cals per day for 7 days reverses T2 diabete...,2015-08-01T10:37:28+00:00,en,N/A,N/A,0,N/A,[http://link.springer.com/article/10.1007/s001...,...,0,1,http://link.springer.com/article/10.1007/s0012...,None,None,None,None,None,0,0
4,681513454931345408,2895499182,Found out there is a Fake Kanji creation conte...,2015-12-28T16:34:04+00:00,en,N/A,N/A,0,N/A,[http://www.sankeisquare.com/event/kanjicontes...,...,0,1,http://www.sankeisquare.com/event/kanjicontest...,None,None,None,None,None,0,0


In [47]:
dfz.shape

(17975, 21)

In [48]:
dfz.checked.sum()

1552

In [49]:
# checked_url = pd.DataFrame(short_url).url.tolist()

In [50]:
domain_list[:10]

[['edition.cnn.com'],
 ['stanford.edu'],
 ['bandedessinee.blog.lemonde.fr'],
 ['link.springer.com'],
 ['www.sankeisquare.com'],
 ['gifthefeeling.com'],
 ['archive.org'],
 ['www.nature.com'],
 ['kite.com'],
 ['www.penguin.co.uk']]

In [51]:
# import json

# with open('short_url.json') as json_file:
#     short_url = json.load(json_file)

# checked_url = pd.DataFrame(short_url).url.tolist()
# # checked_url = []
# # short_url = []
# for c, i in enumerate(dfz.urls):
#     if (c + 1) % 100 == 0:
#         print(f"{(c+1)} / {len(dfz.urls)}")
#     for d in i:
#         if d in checked_url:
#             print(d)
#             pass
#         else:
#             print(c)
#             dd = get_domain(d)
#             e = expand_link(d)
#             de = get_domain(e)
#             short_url.append(
#                 {
#                     "url": d,
#                     "short_url": dd,
#                     "long_url": de,
#                     "is_news": (de in news_domains),
#                     "diff_url": (de == dd),
#                 }
#             )
#     with open("short_url.json", "w") as file:
#         file.write(json.dumps(short_url, indent=4))

In [52]:
dfz.urls[1083:1084]

1083    [https://st.news/2ImPHpV]
Name: urls, dtype: object

In [54]:
checked_url = pd.DataFrame(short_url).url.tolist()

In [56]:
len(checked_url)

1229

In [57]:
import json

with open('short_url.json') as json_file:
    short_url = json.load(json_file)

checked_url = pd.DataFrame(short_url).url.tolist()
# checked_url = []
# short_url = []
x = 1084
skip_those = ["https://st.news/2ImPHpV"]
for c, i in enumerate(dfz.urls[x:]):
    if (c + 1) % 100 == 0:
        print(f"{(c+1)} / {len(dfz.urls)}")
    for d in i:
        if d in checked_url:
            print(d)
            pass
        else:
            print(c)
            dd = get_domain(d)
            e = expand_link(d)
            de = get_domain(e)
            short_url.append(
                {
                    "url": d,
                    "short_url": dd,
                    "long_url": de,
                    "is_news": (de in news_domains),
                    "diff_url": (de == dd),
                }
            )
    with open("short_url.json", "w") as file:
        file.write(json.dumps(short_url, indent=4))

https://proceedings.neurips.cc/paper/2020/file/ef8b5fcc338e003145ac9c134754db71-Paper.pdf
https://www.youtube.com/watch?v=CeeoFqE2OU0
https://www.theglobeandmail.com/canada/article-canada-has-agreement-with-pfizer-to-begin-delivering-early-doses-of/
https://proceedings.neurips.cc/paper/2020/hash/7a685d9edd95508471a9d3d6fcace432-Abstract.html
https://neurips.cc/virtual/2020/protected/poster_7a685d9edd95508471a9d3d6fcace432.html
http://illusionoftheyear.com/vote/
https://bearradio.bandcamp.com/album/cant-buy-yellow
https://chineseliteraturepodcast.com/?p=618
https://ieeexplore.ieee.org/document/9281016
https://bit.ly/2KuQINm
https://bit.ly/smw8dec)
10
11
12
13
14
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
36
37
38
39
40
41
41
42
42
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
65
66
67
68
69
https://bit.ly/38Oe5M1
71
72
73
74
75
76
77
78
79
79
79
80
81
82
82


KeyboardInterrupt: 

In [ ]:
tweet= dfz[dfz.id == 1148208825721393153].reset_index(drop=True)["full_text"][0]

In [ ]:
find_url(tweet)

In [ ]:
grab_title(find_url(tweet)[0])

In [ ]:
%%time
url = "https://lukecarneal.substack.com/p/why-are-some-organic-farmers-turning"
print(HTMLSession().get(url).html.find('title', first=True).text)

In [ ]:
%%time
tw_url = "https://ruthmalan.com/Journal/2019/20190629SlideDocTechnicalLeadershipDecisions.pdf"
res = HTMLSession().get(tw_url)
res.html.find('title', first=True)

In [ ]:
# tw_url = grab_title(find_url(tweet)[0])
# HTMLSession().get(tw_url)
# try:
#     title = HTMLSession().get(tw_url).html.find('title', first=True).text
#     print(title)
# except Exception as ex:
#     print(0)

In [ ]:
import json

with open('short_url.json') as json_file:
    short_url = json.load(json_file)
    
checked_url = pd.DataFrame(short_url).url.tolist() 
    
# short_url = []
x = 59
for c, i in enumerate(dfz.urls[x:]):
    if (c + 1) % 100 == 0:
        print(f"{(c+1)} / {len(dfz.urls)}")
    for d in i:
        if d in checked_url:
            print(d)
            pass
        else:
            print(c)
            dd = get_domain(d)
            e = expand_link(d)
            de = get_domain(e)
            short_url.append(
                {
                    "url": d,
                    "short_url": dd,
                    "long_url": de,
                    "title": grab_title(d),
                    "is_news": (de in news_domains),
                    "diff_url": (de == dd),
                }
            )
    with open("short_url.json", "w") as file:
        file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[1165:1166]

In [ ]:
x = 1166
for c, i in enumerate(dfz.urls[x:]):
    print(c+x, i)
    if (c + x + 1) % 100 == 0:
        print(f"{(c+1)} / {len(dfz.urls)}")
    for d in i:
        dd = get_domain(d)
        e = expand_link(d)
        de = get_domain(e)
        if not (de == dd):
            short_url.append(
                {"url":d, "short_url": dd, "long_url": de, "is_news": (de in news_domains)}
            )
    with open('short_url.json', 'w') as file:
        file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[3573:3574]

In [ ]:
# x = 3574
# for c, i in enumerate(dfz.urls[x:]):
#     print(c+x, i)
#     if (c + x + 1) % 100 == 0:
#         print(f"{(c+1)} / {len(dfz.urls)}")
#     for d in i:
#         dd = get_domain(d)
#         e = expand_link(d)
#         de = get_domain(e)
#         if not (de == dd):
#             short_url.append(
#                 {"url":d, "short_url": dd, "long_url": de, "is_news": (de in news_domains)}
#             )
#     with open('short_url.json', 'w') as file:
#         file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[3586:3587]

In [ ]:
x = 3587
for c, i in enumerate(dfz.urls[x:]):
    print(c+x, i)
    if (c + x + 1) % 100 == 0:
        print(f"{(c+1)} / {len(dfz.urls)}")
    for d in i:
        dd = get_domain(d)
        e = expand_link(d)
        de = get_domain(e)
        if not (de == dd):
            short_url.append(
                {"url":d, "short_url": dd, "long_url": de, "is_news": (de in news_domains)}
            )
    with open('short_url.json', 'w') as file:
        file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[5652:5653]

In [ ]:
bitx = 5653
for c, i in enumerate(dfz.urls[x:]):
    print(c+x, i)
    if (c+x+1) % 100 == 0:
        print(f"{(c+x+1)} / {len(dfz.urls)}")
    for d in i:
        dd = get_domain(d)
        e = expand_link(d)
        de = get_domain(e)
        if not (de == dd):
            short_url.append({"short_url": dd, "long_url": de, "is_news": (de in news_domains)})
    with open('short_url', 'w') as file:
        file.write(json.dumps(short_url, indent=4))